# Engenharia de dados

 - Esse primeiro arquivo tem como foco a estruturação da tabela do projeto. Arquivo chamado data_engineer, pois vamos focar em trazer tudo que for de proparação dos dados para esse arquivo.


## Bibliotecas
 - Centralizei um arquivo para ter todas as bibliotecas necessárias para a engenharia.

In [90]:
import pandas as pd
import numpy as np
import warnings
import json
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
warnings.filterwarnings('ignore')

## Extração de dados - Bases necessárias para o projeto
 - Bases que foram repassadas para fazer o projeto

In [62]:
nb_dir = Path.cwd().parent
data_dir = nb_dir / "data"
df_transcript = pd.read_json(data_dir / "bronze" / "transcript.json", lines=True)
df_profile    = pd.read_json(data_dir / "bronze" / "profile.json",    lines=True)
df_portfolio  = pd.read_json(data_dir / "bronze" / "portfolio.json",  lines=True)


# Visualizar as primeiras linhas
print("Transcript:")
display(df_transcript.head(3))

print("Profile:")
display(df_profile.head(3))

print("Portfolio:")
display(df_portfolio.head(3))


Transcript:


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


Profile:


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


Portfolio:


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed


## Análises iniciais para a parte extrutural da tabela:
 - Para fazer um tratamento de unificar as bases, precisamos mexer com a estrutura do nosso dado.
 - A tabela transcript precisa de um tratamento na coluna value, que a principio tem como seus dados uma tupla.
 - A tabela profile precisa de um tratamento na coluna became_member_on, como é uma informação de data, vamos tratar esse campo.
 - A tabeal portfolio precisa de um tratamento na coluna channels aonde temos seus dados em uma lista.

### Tratamento na _tabela transcript_

In [63]:
def extrai_valores(v):
    if isinstance(v, dict):
        return pd.Series({
            'amount': v.get('amount'),
            'offer_id': v.get('offer_id') or v.get('offer id'),
            'reward': v.get('reward')
        })

    return pd.Series({'amount': np.nan, 'offer_id': np.nan, 'reward': np.nan})



valores = df_transcript['value'].apply(extrai_valores)
df_transcript = pd.concat(
    [df_transcript.drop(columns=['value']), valores],
    axis=1
)


### Tratamento na _tabela portifolio_

In [64]:
df_portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


Como verificamos na base portifolio, temos uma variavel channels e vamos tentar quebrar ela para trabalhar melhor com ela.
 - Vamos utilizar a biblioteca do sklearn de processamento, MultiLabelBinarizer 

In [65]:
mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(df_portfolio['channels'])
df_encoded = pd.DataFrame(encoded, columns=mlb.classes_)
df_portfolio = pd.concat([df_portfolio, df_encoded], axis=1)
df_portfolio = df_portfolio.drop(columns={'channels'},axis=1)



In [66]:
df_portfolio

,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


### Tratamento _tabela profile_

Tratamentos simples:

 - Coluna became_member_on: Aonde vamos transformar em datetime.
 - Coluna event: campos padronizados

In [67]:
df_profile['became_member_on'] = pd.to_datetime(
    df_profile['became_member_on'].astype(str),
    format='%Y%m%d',
    errors='coerce')


### Unificando tabelas

In [68]:
master = df_transcript.merge(
    df_profile,
    left_on= 'person', right_on='id',
    how='left'
)

print(master.columns)
master = master.merge(
    df_portfolio,
    left_on='offer_id', right_on='id',
    how='left',
    suffixes=('', '_offer')
)

print(master.head(3))

Index(['person', 'event', 'time', 'amount', 'offer_id', 'reward', 'gender',
       'age', 'id', 'became_member_on', 'income'],
      dtype='object')
                             person           event  time  amount  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received     0     NaN   
1  a03223e636434f42ac4c3df47e8bac43  offer received     0     NaN   
2  e2127556f4f64592b11af22de27a7932  offer received     0     NaN   

                           offer_id  reward gender  age  \
0  9b98b8c7a33c4b65b9aebfe6a799e6d9     NaN      F   75   
1  0b1e1539f2cc45b7b9fa7c272da2e1d7     NaN   None  118   
2  2906b810c7d4411798c6938adc9daaa5     NaN      M   68   

                                 id became_member_on    income  reward_offer  \
0  78afa995795e4d85b5d9ceeca43f5fef       2017-05-09  100000.0           5.0   
1  a03223e636434f42ac4c3df47e8bac43       2017-08-04       NaN           5.0   
2  e2127556f4f64592b11af22de27a7932       2018-04-26   70000.0           2.0   

   difficulty  d

In [69]:
caminho_dic = nb_dir / "utils" / "dic.json"
with open(caminho_dic, "r", encoding="utf-8") as f:
    dados_dic = json.load(f)

eventos = dados_dic["eventos"]

def listagem_dados(dados):
    dicionario = {}
    for dados, itens in dados.items():
        
        for item in itens:
            dicionario[item] = dados
    return dicionario

eventos_padronizados = listagem_dados(eventos)


### Tratamento _tabela master_

In [70]:
master['event'] = (
    master['event']
      .map(eventos_padronizados)                     
      .fillna(master['event'])
)

In [71]:
master = master.drop(columns={'id','id_offer'})
master

,person,event,time,amount,offer_id,reward,gender,age,became_member_on,income,reward_offer,difficulty,duration,offer_type,email,mobile,social,web
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,F,75,2017-05-09,100000.0,5.0,5.0,7.0,bogo,1.0,1.0,0.0,1.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,None,118,2017-08-04,NaN,5.0,20.0,10.0,discount,1.0,0.0,0.0,1.0
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,M,68,2018-04-26,70000.0,2.0,10.0,7.0,discount,1.0,1.0,0.0,1.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,None,118,2017-09-25,NaN,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,None,118,2017-10-02,NaN,10.0,10.0,5.0,bogo,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN,NaN,M,66,2018-01-01,47000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN,NaN,M,52,2018-04-08,62000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN,NaN,F,63,2013-09-22,52000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN,NaN,M,57,2016-07-09,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Precisamos tratar os casos que são transaction porque no caso seria transaction os casos que foram completos. Então fizemos essa alteração nos dados finais.

In [72]:
df_master = master.copy() 

tx = (
    df_master[df_master['event'] == 'transaction'][['person', 'time', 'amount']]
    .rename(columns={'amount': 'amount_txn'})
)

completed = df_master[df_master['event'] == 'offer completed'][['person', 'time', 'offer_id']]

matched = completed.merge(tx, on=['person', 'time'], how='left')

df_master = df_master.merge(matched[['person', 'time', 'amount_txn']], on=['person', 'time'], how='left')

df_master['amount'] = np.where(df_master['event'] == 'offer completed', df_master['amount_txn'], np.nan)

df_master.drop(columns=['amount_txn'], inplace=True)
df_master = df_master[df_master['event']!='transaction']

In [88]:
clientes_sen_oferta = pd.merge(df_master, master, on='person', how='outer', indicator=True )
clientes_sen_oferta = clientes_sen_oferta[clientes_sen_oferta['_merge']=='right_only']

## Salvando tabela silver

In [89]:
df_master.to_csv(data_dir/ "silver" /'dados_finais.csv', index=False)
clientes_sen_oferta.to_csv(data_dir/ "silver" /'dados_clientes_sem_oferta.csv', index=False)